# Process single cell morphology features for CellProfiler readouts

## Import Libraries

In [1]:
import pathlib
import pandas as pd

from pycytominer.cyto_utils import cells, output

## Set up paths to CellProfiler directory and outputs

In [2]:
# Set file and directory constants
cp_dir = "../2.cellprofiler_analysis"
output_dir = "data"

## Set paths to sqlite files

In [3]:
# Set name and path of .sqlite file and path to metadata
sql_file = "interstellar_wave1_dilate50.sqlite"
single_cell_file = f"sqlite:///{cp_dir}/analysis_output/{sql_file}"
platemap_file = f"{cp_dir}/metadata/Interstellar_wave1_platemap.csv"

# Set path with name for outputted data
sc_output_file = pathlib.Path(f"{output_dir}/interstellar_wave1_dilate50_sc.csv.gz")

## Set up names for linking columns between tables in the database file

In [4]:
# Define custom linking columns between compartments
linking_cols = {
    "Per_TranslocatedNuclei": {
        "Per_DilatedNuclei": "TranslocatedNuclei_Parent_DilatedNuclei",
        "Per_Nuclei": "TranslocatedNuclei_Parent_Nuclei",
    },
    "Per_DilatedNuclei": {"Per_TranslocatedNuclei": "DilatedNuclei_Number_Object_Number"},
    "Per_Nuclei": {"Per_TranslocatedNuclei": "Nuclei_Number_Object_Number"},
}

## Load and view platemap file

In [5]:
# Load platemap file
platemap_df = pd.read_csv(platemap_file)
platemap_df.head()

,well,wellName,row,col,alias,treatment,dose
0,A1,A1,1,1,1,LPS,10µg/ml
1,B1,B1,2,1,2,LPS,1µg/ml
2,C1,C1,3,1,3,H2O2,500µM
3,D1,D1,4,1,4,H2O2,50µM
4,E1,E1,5,1,5,ATP,1mM


## Set up `SingleCells` class from Pycytominer

In [6]:
# Instantiate SingleCells class
sc = cells.SingleCells(
    sql_file=single_cell_file,
    compartments=["Per_DilatedNuclei", "Per_TranslocatedNuclei", "Per_Nuclei"],
    compartment_linking_cols=linking_cols,
    image_table_name="Per_Image",
    strata=["Image_Metadata_Well", "Image_Metadata_Plate"],
    merge_cols=["ImageNumber"],
    image_cols="ImageNumber",
    load_image_data=True
)

/home/jenna/anaconda3/envs/3.process_features_interstellar/lib/python3.8/site-packages/pycytominer/cyto_utils/util.py:61: UserWarning: Non-canonical compartment detected: per_dilatednuclei, per_translocatednuclei, per_nuclei
  warnings.warn(warn_str)


## Merge single cells 

In [7]:
# Merge single cells across compartments
anno_kwargs = {"join_on": ["Metadata_well", "Image_Metadata_Well"]}

sc_df = sc.merge_single_cells(
    platemap=platemap_df,
    **anno_kwargs,
)

# Save level 2 data as a csv
output(sc_df, sc_output_file)

print(sc_df.shape)
sc_df.head()

(17561, 841)


,Metadata_wellName,Metadata_row,Metadata_col,Metadata_alias,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_TranslocatedNuclei_Parent_DilatedNuclei,...,Nuclei_Texture_SumVariance_CorrGasderminD_3_02_256,Nuclei_Texture_SumVariance_CorrGasderminD_3_03_256,Nuclei_Texture_Variance_CorrDNA_3_00_256,Nuclei_Texture_Variance_CorrDNA_3_01_256,Nuclei_Texture_Variance_CorrDNA_3_02_256,Nuclei_Texture_Variance_CorrDNA_3_03_256,Nuclei_Texture_Variance_CorrGasderminD_3_00_256,Nuclei_Texture_Variance_CorrGasderminD_3_01_256,Nuclei_Texture_Variance_CorrGasderminD_3_02_256,Nuclei_Texture_Variance_CorrGasderminD_3_03_256
0,A10,1,10,9,Disulfiram,2.5µM,19,70117_20230118MM1_Ab Test_V2,A10,1,...,0.562764,0.519955,5.453863,5.338515,5.496212,5.341068,0.208618,0.20586,0.207914,0.20388
1,A10,1,10,9,Disulfiram,2.5µM,19,70117_20230118MM1_Ab Test_V2,A10,4,...,0.562764,0.519955,5.453863,5.338515,5.496212,5.341068,0.208618,0.20586,0.207914,0.20388
2,A10,1,10,9,Disulfiram,2.5µM,19,70117_20230118MM1_Ab Test_V2,A10,2,...,0.606445,0.583598,1.243379,1.224368,1.241659,1.238014,0.248763,0.246857,0.247815,0.248309
3,A10,1,10,9,Disulfiram,2.5µM,19,70117_20230118MM1_Ab Test_V2,A10,3,...,0.630966,0.607802,1.578343,1.586902,1.568943,1.577166,0.249089,0.249433,0.24949,0.249476
4,A10,1,10,9,Disulfiram,2.5µM,19,70117_20230118MM1_Ab Test_V2,A10,5,...,0.635611,0.585116,3.693021,3.61539,3.75539,3.691136,0.240287,0.240656,0.24094,0.242418


## View info of the dataframe

In [8]:
sc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17561 entries, 0 to 17560
Columns: 841 entries, Metadata_wellName to Nuclei_Texture_Variance_CorrGasderminD_3_03_256
dtypes: int64(3), object(838)
memory usage: 112.8+ MB


---

### Visualize basic count statistics

In [9]:
sc_df.Metadata_treatment.value_counts()

Thapsi             4241
LPS + Nigericin    4160
Disulfiram         3853
DMSO 0.1%          2671
Media only         2636
Name: Metadata_treatment, dtype: int64

In [10]:
pd.crosstab(sc_df.Metadata_treatment, sc_df.Metadata_Well)

Metadata_Well,A10,B10,C10,D10,E10,F10,G10,H10
Metadata_treatment,,,,,,,,
DMSO 0.1%,0,0,0,0,0,0,2671,0
Disulfiram,1356,2497,0,0,0,0,0,0
LPS + Nigericin,0,0,0,0,1659,2501,0,0
Media only,0,0,0,0,0,0,0,2636
Thapsi,0,0,1717,2524,0,0,0,0
